In [1]:
import os
%pwd

'c:\\Users\\49179\\Desktop\\Food_image_classification\\research'

In [2]:
os.chdir("../")

os.getcwd()

'c:\\Users\\49179\\Desktop\\Food_image_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path
import torch


@dataclass
class training_config:
    root_dir: Path
    model_path: Path
    epochs: int
    batch_size: int
    learning_rate: float
    device: str



In [4]:
from Food_Classification.utils.common import read_yaml, create_directory
from Food_Classification.constants import *

class ConfigurationManager:
    def __init__(self, 
                 config_file_path= CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directory([self.config.artifacts_root])


    def get_training_config(self) -> training_config:
        config = self.config.model_training
        create_directory([config.root_dir])
        self.model_path = self.config.prepare_base_model.updated_base_model
        Training_Config = training_config(root_dir= config.root_dir,
                                          model_path= self.model_path,
                                          epochs= self.params.EPOCHS,
                                          batch_size= self.params.BATCH_SIZE,
                                          learning_rate= self.params.LEARNING_RATE,
                                          device=self.params.DEVICE
                                         )


        return Training_Config



In [5]:
#%%writefile src/Food_Classification/components/model_trainier.py
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter
import torch
from Food_Classification import logger
from Food_Classification.config.configuration import training_config
from Food_Classification.entity.artifact_entity import DataTransformationArtifact



class Model_training:
    def __init__(self, config: training_config, 
                        artifact: DataTransformationArtifact,
                        loss_function: torch.nn.Module, 
                        optimizer: torch.optim.Optimizer, 
                        writer: SummaryWriter):
        self.config = config
        self.artifact: DataTransformationArtifact = artifact
        self.train_dataloader = self.artifact[0]
        self.test_dataloader = self.artifact[1]
        self.model = torch.load(self.config.model_path,map_location=torch.device(self.config.device))
        self.loss_function = loss_function
        self.optimizer = optimizer
        self.writer = writer
    
    def train_step(self):
        model = self.model
        optimizer = self.optimizer(params= self.model.parameters(),lr = self.config.learning_rate)
        try:
            # Initiate model training
            model.train()
            train_loss = 0
            train_acc= 0
            progress = tqdm(self.train_dataloader)
            # Looping through batches
            for batch, (data, target) in enumerate(progress):
                # Setting up data into device
                data, target = data.to(self.config.device), target.to(self.config.device)

                # forward propogation
                target_pred = model(data)

                # Calculating loss
                loss = self.loss_function(target_pred, target)
                train_loss += loss.item()
                
                # setting optimizer to zero gradient
                optimizer.zero_grad()
                
                # backward propogation
                loss.backward()
                
                # updating weights
                optimizer.step()


                
                # Calculating accuracy
                target_pred_class = torch.argmax(torch.softmax(target_pred, dim=1), dim=1) 
                train_acc += (target_pred_class == target).sum().item() / len(target_pred)

            train_loss /= len(self.train_dataloader)
            train_acc /= len(self.train_dataloader)
            progress.set_description(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
            return train_loss, train_acc
                
            
            
        except Exception as e:
            raise e


    def test_step(self):
        model = self.model
        try:
            # Intiatig model evaluation
            model.eval()

            test_loss, test_acc = 0,0
            progress = tqdm(self.test_dataloader)

            with torch.inference_mode():
                for batch, (data, target) in enumerate(progress):
                    data, target = data.to(self.config.device), target.to(self.config.device)

                    # forward propogation
                    target_pred = model(data)

                    # Calculating Loss
                    loss = self.loss_function(target_pred, target)
                    test_loss += loss.item()

                    # calculating Accuracy
                    target_pred_labels = target_pred.argmax(dim=1)
                    test_acc += ((target_pred_labels) == target).sum().item()/len(target_pred_labels)

            test_loss = test_loss/len(self.test_dataloader)
            test_acc = test_acc /len(self.test_dataloader)
            return test_loss, test_acc
        except Exception as e:
            raise e


    def initiate_training(self):
        model = self.config.model_path
        results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
        }

        for epoch in tqdm(range(self.config.epochs)):
            train_loss, train_acc = self.train_step()
            test_loss, test_acc = self.test_step()

            logger.info(f"Epoch: {epoch+1} | "
                f"self.train_loss: {train_loss:.4f} | "
                f"self.train_acc: {train_acc:.4f} | "
                f"self.test_loss: {test_loss:.4f} | "
                f"self.test_acc: {test_acc:.4f}")

            # Update results dictionary
            results["train_loss"].append(train_loss)
            results["train_acc"].append(train_acc)
            results["test_loss"].append(test_loss)
            results["test_acc"].append(test_acc)

        # Save the model
        trained_model = os.path.join(self.config.root_dir, "trained_model.pth")
        torch.save(model,trained_model)

         ### Experiment tracking
        self.writer.add_scalars(main_tag="Loss",
                            tag_scalar_dict = {"Train_loss" : train_loss,
                                                "Test_loss" : test_loss},
                            global_step= epoch
                            )
        self.writer.add_scalars(main_tag="Accuracy",
                            tag_scalar_dict = {"Train_acc" : train_acc,
                                                "Test_acc" : test_acc},
                            global_step= epoch
                            )

       # self.writer.add_graph(model=model,
       # input_to_model = torch.randn(32,3,224,224).to(self.config.device))
    # Return the filled results at the end of the epochs
        return results

c:\Users\49179\anaconda3\envs\Food\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from Food_Classification.components.data_transformation import data_transformation
from Food_Classification.components.tensorboard import preparetensorboard
from Food_Classification.components.prepare_base_model import PrepareBaseModel
from Food_Classification.config.configuration import ConfigurationManager
config = ConfigurationManager()
transformation_config = config.get_data_transform_config()
transform = data_transformation(config=transformation_config)
data_transformation_artifact = transform.initiate_data_transformation()
tensorboard = preparetensorboard(config=config.get_tensorboard_config())
writer = tensorboard.get_summary_writer()
prepare_base_model_config = config.get_base_model_config()
prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
prepare_base_model.get_base_model()
prepare_base_model.update_base_model()
train_config = config.get_training_config()
training = Model_training(train_config, artifact=data_transformation_artifact, loss_function= torch.nn.CrossEntropyLoss(), optimizer= torch.optim.Adam,writer=writer)
training.initiate_training()

[2024-05-07 21:13:34,544: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-05-07 21:13:34,547: INFO: common: yaml file params.yaml loaded successfully]
[2024-05-07 21:13:34,547: INFO: common: directory artifacts created successfully]
[2024-05-07 21:13:34,548: INFO: common: directory artifacts/data_transforms created successfully]
[2024-05-07 21:13:34,549: INFO: data_transformation: Initiating data transformation]
[2024-05-07 21:13:34,549: INFO: data_transformation: Transforming train data]
[2024-05-07 21:13:34,550: INFO: data_transformation: Transforming test data]
[2024-05-07 21:13:34,554: INFO: data_transformation: creating_dataloaders]
[2024-05-07 21:13:34,589: INFO: data_transformation: DataLoaders created]
[2024-05-07 21:13:34,590: INFO: common: directory artifacts/prepare_tensorboard created successfully]
[2024-05-07 21:13:34,591: INFO: common: directory artifacts/prepare_tensorboard/tensorboard_log_dir created successfully]
[2024-05-07 21:13:34,595: INFO: co

100%|██████████| 63/63 [01:13<00:00,  1.16s/it]


[2024-05-07 21:16:27,271: INFO: 3855471529: Epoch: 1 | self.train_loss: 2.6828 | self.train_acc: 0.2924 | self.test_loss: 2.3945 | self.test_acc: 0.4529]


100%|██████████| 1/1 [02:51<00:00, 171.32s/it]


{'train_loss': [2.6827739616657826],
 'train_acc': [0.2923869680851064],
 'test_loss': [2.394515571140108],
 'test_acc': [0.45287698412698413]}

In [19]:
%%writefile src\Food_Classification\pipeline\stage_4_Model_training_pipeline.py
from Food_Classification.config.configuration import ConfigurationManager
from Food_Classification.components.tensorboard import preparetensorboard
from Food_Classification.components.model_trainier import Model_training
from Food_Classification import logger

STAGE_NAME = "Model Training"

class ModelTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        try:
            config = ConfigurationManager()
            tensorboard = preparetensorboard(config=config.get_tensorboard_config())
            writer = tensorboard.get_summary_writer()
            train_config = config.get_training_config()
            training = Model_training(train_config, artifact=data_transformation_artifact, loss_function= torch.nn.CrossEntropyLoss(), optimizer= torch.optim.Adam,writer=writer)
            training.initiate_training()
        except Exception as e:
            raise e
        

if __name__ == "__main__":
    try:
        logger.info(f">>>>>>> stage : {STAGE_NAME} <<<<<<<<")
        Preapare_base_model = ModelTrainingPipeline()
        Preapare_base_model.main()
        logger.info(f">>>>>>> stage : {STAGE_NAME} completed <<<<<<<< \n\nx========x")

    except Exception as e:
        logger.exception(e)
        raise e


Overwriting src\Food_Classification\pipeline\stage_4_Model_training_pipeline.py
